# 🍎 SKT Risk Management (Raf Ömrü Analizi)

Bu çalışma, **Son Kullanma Tarihi (SKT)** yaklaşan ürünlerin satış performansını ve fire riskini analiz eder.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

df = pd.read_csv('../data/mock_sales_data.csv')
df['date'] = pd.to_datetime(df['date'])
df['expiry_date'] = pd.to_datetime(df['expiry_date'])
df.head()

## 1. Feature Engineering: Kalan Raf Ömrü (RSL)
Satış anında ürünün kaç günü kalmıştı?

In [ ]:
df['days_to_expire'] = (df['expiry_date'] - df['date']).dt.days

# Negatif değerleri (zaten tarihi geçmiş) filtreleyelim veya 0 kabul edelim (simülasyon hatası temizliği)
df = df[df['days_to_expire'] >= 0]

sns.histplot(df['days_to_expire'], bins=30)
plt.title('Satılan Ürünlerin Kalan Raf Ömrü Dağılımı')
plt.show()

## 2. SKT Yaklaştıkça İndirim Oranı
RSL azaldıkça promosyon (indirim) artırılıyor mu?

In [ ]:
# RSL grupları oluşturalım
df['rsl_bucket'] = pd.cut(df['days_to_expire'], bins=[0, 7, 15, 30, 90, 365], labels=['0-7 Gün', '7-15 Gün', '15-30 Gün', '1-3 Ay', '+3 Ay'])

avg_price_by_rsl = df.groupby('rsl_bucket')['unit_price'].mean().reset_index()
print(avg_price_by_rsl)

sns.barplot(data=avg_price_by_rsl, x='rsl_bucket', y='unit_price')
plt.title('Kalan Ömür Grubuna Göre Ortalama Satış Fiyatı')
plt.show()

## 3. Karar Mekanizması
Eğer `0-7 Gün` kaldıysa ve satış hızı düşükse -> **Otomatik %50 İndirim Öner.**